In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data_folder_path = "MMSR_WT22_Task1_Data"

In [3]:
info_df = pd.read_csv(os.path.join(data_folder_path, "id_information_mmsr.tsv"), delimiter='\t')
genres_df = pd.read_csv(os.path.join(data_folder_path, "id_genres_mmsr.tsv"), delimiter='\t')

tfidf_df = pd.read_csv(os.path.join(data_folder_path, "id_lyrics_tf-idf_mmsr.tsv"), delimiter='\t')
tfidf_array = tfidf_df.to_numpy()

word2vec_df = pd.read_csv(os.path.join(data_folder_path, "id_lyrics_word2vec_mmsr.tsv"), delimiter='\t')
word2vec_array = word2vec_df.to_numpy()

bert_df = pd.read_csv(os.path.join(data_folder_path, "id_bert_mmsr.tsv"), delimiter='\t')
bert_array = bert_df.to_numpy()

In [4]:
display(info_df.iloc[1:2])
display(genres_df.iloc[0:1])
display(tfidf_df.iloc[0:1])
display(word2vec_df.iloc[0:1])
display(bert_df.iloc[0:1])

,id,artist,song,album_name
1,0010xmHR6UICBOYT,Oddisee,After Thoughts,The Beauty in All


,id,genre
0,0009fFIM1eYThaPg,['pop']


,id,abl,accept,across,act,addict,afraid,age,ago,ah,...,yea,yeah,year,yellow,yes,yesterday,yet,yo,young,youth
0,9jbSytob9XRzwvB6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.150511


,id,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,0LiOzxBZ1aPLlFsK,0.031109,0.018026,0.022785,0.028802,-0.026084,-0.006278,0.030599,-0.041043,0.036703,...,-0.025845,0.010468,-0.047819,0.00562,-0.025106,-0.017939,-0.009981,-0.027846,0.0211,-0.020994


,id,0,1,2,3,4,5,6,7,8,...,758,759,760,761,762,763,764,765,766,767
0,9jbSytob9XRzwvB6,0.009225,0.041393,-0.003659,-0.0305,-0.006346,-0.019719,-0.075958,-0.003737,-0.000486,...,-0.037228,-0.027013,0.029167,0.009537,-0.010819,0.004054,-0.018671,0.012545,0.020696,-0.019794


In this first task, please implement a simple reusable(!) framework for text-based retrieval/similarity of music pieces (we will use the term “song” and “track” synonymously in the following). The input (query) is a song, more precisely its meta-data, i.e., artist and track name. The output of the system should be a list of songs that are similar to the query song. Throughout the practical part you will investigate various ways to define this similarity. For this first exercise, only consider textual representations of the song, in particular lyrics features (i.e., TF-IDF, word2vec, and BERT embeddings). Start with something simple (e.g., cosine similarity computed on TF-IDF vectors); and then add one additional variant (i.e., combination of song representation and similarity metric).

In [5]:
tfidf_array

array([['9jbSytob9XRzwvB6', 0.0, 0.0, ..., 0.0, 0.0, 0.1505109354143906],
       ['Njp6JPM8vitbhVJU', 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       ['h48f46ZsT9h0Z5Dm', 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       ...,
       ['76qenAf8gYlH4pfq', 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       ['9F8jQjeibAuZinEP', 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       ['njEze6oZPy3MX2dN', 0.0, 0.0, ..., 0.0, 0.0, 0.0]], dtype=object)

In [6]:
# Similarity:
#   1. load datasets --> pd
#   2. extract vector: pd --> np (except alphanumerical id)
#   3. pairwise similarity 
def filter_df(df, **args):
    if not args: 
        raise AttributeError("**args required")
    query = ''
    for k, v in args.items():
        query += f"{k}=='{v}' and "
    query = query[:-5]
    return df.query(query)

In [7]:
example_artist = "Cheryl"
example_song = "Rain on Me"
example_embedding_df = bert_df
example_embedding_array = bert_array

In [8]:
example_id = filter_df(info_df, artist=example_artist, song=example_song).to_numpy()[0, 0]
print(f"example_song found by artist='{example_artist}' and song='{example_song}' --> id='{example_id}'")

example_song found by artist='Cheryl' and song='Rain on Me' --> id='0009fFIM1eYThaPg'


In [9]:
example_Y = filter_df(example_embedding_df, id=example_id).to_numpy().reshape(1, -1)
cs = cosine_similarity(X=example_embedding_array[:, 1:], Y=example_Y[:, 1:])
print("avg_similarity (all songs):", np.mean(cs))

avg_similarity (all songs): 0.3988439414994516


In [10]:
example_X = filter_df(info_df.merge(example_embedding_df), artist=example_artist).to_numpy()[:, 3:]
cs = cosine_similarity(X=example_X[:, 1:], Y=example_Y[:, 1:])
print("similarities (within example_artist):\n", cs)

similarities (within example_artist):
 [[1.        ]
 [0.45588829]
 [0.415266  ]
 [0.56247426]
 [0.52213525]
 [0.54025452]
 [0.58719805]
 [0.51434849]
 [0.48975459]
 [0.50849349]
 [0.40145441]
 [0.48441612]
 [0.45773557]
 [0.49066422]]


In [11]:
def top_k_similar(id, embedding_array, k=None, similarity_measure=cosine_similarity):
    similarities = similarity_measure(X=embedding_array[:, 1:], Y=embedding_array[embedding_array[:, 0]==id].reshape(1, -1)[:, 1:]).reshape(-1)
    most_similar = [[this_id, this_sim] for this_sim, this_id in sorted(zip(similarities, embedding_array[:, 0]))][::-1]
    if k is None:
        return most_similar
    return most_similar[1:k+1] # return top k but strip self

In [12]:
top_k_similar(example_id, example_embedding_array, k=5, similarity_measure=cosine_similarity)

[['WAFIWIziIPINi0MC', 0.8294106047397072],
 ['lbTW6YAzKbARhB59', 0.8273032882018981],
 ['NGhsmk5BSCAoQijc', 0.821263876532285],
 ['wXcMv63aWS4KEPm8', 0.8183482638952275],
 ['HFLuvJXc6SjcJt7d', 0.8109073542430318]]